<div align='center'><font size="5" color='#353B47'>Opracowanie aplikacji do wyznaczania giełdowych decyzji inwestycyjnych opartej na analizie Big Data</font></div>
<div align='center'><font size="4" color="#353B47">Projekt Inżynierski</font></div>
<br>
<hr>

**<font color="#5963ab" size="4">Cel projektu</font>**

> Celem projektu jest opracowanie narzędzia do wspierania wyznaczania trendów na podstawie różnych danych dostępnych w internecie. Umożliwi to inwestorom podejmowanie bardziej trafnych i opartych na danych decyzji inwestycyjnych.

## <div id="summary">Spis Treści</div>

**<br><font size="2"><a href="#chap1">Konfiguracja środowiska</a></font>**
**<br><font size="2"><a href="#chap2">Wczytanie danych</a></font>**
**<br><font size="2"><a href="#chap3">Analiza danych</a></font>**
**<br><font size="2"><a href="#chap4">Prztwarzanie wstępne danych</a></font>**
**<br><font size="2"><a href="#chap5">Stowrzenie modeli</a></font>**
**<br><font size="2"><a href="#chap6">Backtesting modeli i optymalizacja</a></font>**
**<br><font size="2"><a href="#chap7">Ocena modeli</a></font>**
**<br><font size="2"><a href="#chap8">Zastosowanie wyników w rzeczywistości</a></font>**
**<br><font size="2"><a href="#chap9">Wnioski z uzyskanych danych</a></font>**
**<br><font size="2"><a href="#chap10">Raport końcowy</a></font>**


<div id="chap1"><h1 style="color:white;background:#5963ab;border-radius:5px;padding:30px;font-family:'Arial', cursive;font-size:50px;text-align:center">Konfiguracja środowiska </h1></div>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# visualization
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# import data
!pip install yfinance
import yfinance as yf

# data analysis
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf,month_plot,quarter_plot
from statistics import mean

# text formatting
from pprint import pprint

# natural language processing
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from transformers import pipeline
from sklearn.feature_extraction.text import CountVectorizer

# machine learning


<div id="chap2"><h1 style="color:white;background:#5963ab;border-radius:5px;padding:30px;font-family:'Arial', cursive;font-size:50px;text-align:center">Wczytanie danych</h1></div>



**<font color="#5963ab" size="4">Jakie dane są wykorzystane i ich pochodzenie.</font>**

>Oto dane użyte w tym notatniku, oraz ich pochodzenie:
>* **Notowania S&P 500**  - pobrane z użyciem biblioteki *yfinance*
>* **Nagłówki artykułów z New York Times**  - pobrane za pomocą API dostarczanego przez NYT - https://developer.nytimes.com/
>* **Dane makroekonomiczne gospodarki USA i podstawowe wskaźniki dla S&P 500**  - pobrane z:
>  * https://data.nasdaq.com/
>  * https://fred.stlouisfed.org/
>  * https://www.bls.gov/
>  * https://stooq.pl/
>* **Wyszukiwanie frazy "S&P 500" w wyszukiwarce** - pobrano dane z wyszukiwarki google ze strony - https://trends.google.com/trends/
>
>Powyższe dane są w dalszych etapach poddane analizie oraz wstępnie przetworzone w celu wykorzystania ich a modelach uczenia maszynowego.
>
>W tym notatniku skupiono się na danych związancyh z indeksem S&P 500, gospodarką Stanów Zjednoczonych, nagłówkami artykułów z NYT, trendami wyszukiwania frazy "S&P 500" w wyszukiwarce google. Na podstawie danych z notowań wyznaczono też wiele wskazników pomocnych przy analizie technicznej, których także użyto do predykcji notowań. Dane pochodzą z okresu od 01/01/2004 do 08/31/2023.

**<font color="#5963ab" size="4">Notowania S&P 500</font>**

In [ ]:
# get S&P 500 financial datadata
try:
    # fetch from yfinance
    sp500 = yf.Ticker("^GSPC")
    
    #print all index info
    #pprint("Info")
    #pprint(sp500.info)

    #get stock quotes
    sp500 = sp500.history( period="max" )
    start_date = '2004-01-01'
    end_date = '2023-08-31'
    sp500 = sp500[(sp500.index >= start_date) & (sp500.index <= end_date)]
except:
    # load the dataset
    sp500 = pd.read_csv('/kaggle/input/s-and-p-500-stock-quotes/SP500_Stock_Quotes.csv', index_col='date')
    sp500.index = pd.DatetimeIndex(sp500.index, tz='America/New_York')
    

# make all column headers and index name lowercase
sp500.columns = sp500.columns.str.lower()
sp500.index.name = sp500.index.name.lower()

sp500

**<font color="#5963ab" size="4">Nagłówki New York Times </font>**

In [ ]:
# load the dataset
nyt_hl = pd.read_csv('/kaggle/input/new-york-times-headlines-news/NYT_Headlines.csv', index_col='date',  encoding='utf-8')
nyt_hl.index = pd.to_datetime(nyt_hl.index)
nyt_hl.index = nyt_hl.index.tz_localize('America/New_York')
nyt_hl

**<font color="#5963ab" size="4">Dane makroekonomiczne gospodarki USA i podstawowe wskaźniki dla S&P 500</font>**

In [ ]:
# load the dataset
macro_data = pd.read_csv('/kaggle/input/usa-sp500-fundamentals/USA_SP500_Fundamentals_2004_2023_Combined.csv', index_col='date')
macro_data.index = pd.to_datetime(macro_data.index)
# make all column headers lowercase
macro_data.columns = macro_data.columns.str.lower()
macro_data.index = macro_data.index.tz_localize('America/New_York')
macro_data

In [ ]:
# divide data to two data frames
# data related with S&P 500 
sp500_rates = macro_data[['pe', 'cape', 'earn_yield','pbv', 'price_sales_ratio','div_yield', 'div_growth']]
sp500_rates

In [ ]:
# data related with USA ecomony
usa_eco_rates = macro_data[['gdp', 'gdp_percent_change', 'unemploy_rate','pmi_index', 'cpiaucsl_pch','fed_effective_rate', 'fed_effect_rate_pch','10usy_bond_yield', 'cons_sent',
       'cons_sent_percent_change']]
usa_eco_rates

**<font color="#5963ab" size="4">Trendy wyszukiwania "S&P 500" w wyszukiwarce google</font>**

In [ ]:
# load the dataset
goog_trend = pd.read_csv('/kaggle/input/s-and-p500-google-trends/SP500_GoogleTrends.csv', index_col='date')
goog_trend.index = pd.to_datetime(goog_trend.index)
goog_trend.index = goog_trend.index.tz_localize('America/New_York')
goog_trend

<div id="chap3"><h1 style="color:white;background:#5963ab;border-radius:5px;padding:30px;font-family:'Arial', cursive;font-size:50px;text-align:center">Analiza danych</h1></div>

**<font color="#5963ab" size="4">Dane z szeregów czasowych.</font>**

>W danym notatniku mamy do czynienia z danymi z szeregów czasowych. Dane z szeregów czasowych (ang. "time series data") to rodzaj danych, w którym informacje są zbierane, zapisywane lub pomiarowe w określonych momentach w czasie lub w regularnych odstępach czasu. Te dane reprezentują zmienne, które ewoluują w czasie i pozwalają na analizę, modelowanie i prognozowanie przyszłych wartości na podstawie wcześniejszych obserwacji. 
>
>Dane z szeregów czasowych mają pewne charakterystyczne cechy, takie jak sezonowość, trendy, cykliczność i autokorelacja, które można wykorzystać do analizy i modelowania. Analiza danych szeregów czasowych może pomóc w identyfikacji wzorców, prognozowaniu przyszłych notowań, zarządzaniu ryzykiem i podejmowaniu właściwych decyzji opartych na danych.






**<font color="#5963ab" size="4">sp500</font>**




>Indeks **S&P 500**, pełna nazwa "Standard & Poor's 500 Index", jest jednym z najważniejszych indeksów giełdowych na świecie. Jest on używany do pomiaru ogólnej wydajności amerykańskiego rynku akcji. Obejmuje on 500 największych i najbardziej znaczących spółek notowanych na giełdzie amerykańskiej, reprezentujących różne sektory gospodarki.
>
>S&P 500 jest szeroko stosowany jako wskaźnik, który pomaga inwestorom i analitykom śledzić ogólną kondycję rynku akcji w Stanach Zjednoczonych. W skład indeksu wchodzą firmy takie jak Apple, Microsoft, Amazon, Google (Alphabet), czy Berkshire Hathaway.
>
>Indeks S&P 500 jest ważnym punktem odniesienia dla wielu inwestorów i profesjonalistów branży finansowej i jest używany do oceny trendów giełdowych, porównywania wyników portfeli inwestycyjnych oraz analizy ogólnego klimatu na rynku.

In [ ]:
# check for missing values in each column
missing_columns = sp500.isnull().sum()
pprint("Missing values in column")
pprint(missing_columns)
# print all sp500 data info
print()
pprint(sp500.info())


In [ ]:
# plot prices
sp500.plot.line(y="close", use_index = True)


>Nie wydaje się, aby dane wykazały jakąkolwiek zauważalną sezonowość w poszczególnych latach. Zamiast tego pokazuje trend wzrostowy, a nie zachowanie cykliczne. Można zauważyć też kilka większych spadków. W 2008r. spadek spowodowany przez światowy kryzys. Na początku 2020 zauważalny spadek cen był związany z pandemią Covid-19. Ostatni widoczny spadek był widoczny w 2022r.. Ogólnie widać bardzo duży wzrost w wartości indeksu w przeciągu niecałych 20 lat.`


In [ ]:
# plot volume
volume_plot = sp500.plot.line(y="volume", use_index = True)
volume_plot.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: '%1.1fB' % (x * 1e-9)))
volume_plot.set_xlabel("Date")
volume_plot.set_ylabel("Volume")
volume_plot.set_title('Volume')

>Wykres wykazuje znaczny szum wynikający z zastosowania skali dziennej. Wybierając średni miesięczny wolumen obrotu, a nie dzienny wolumen obrotu, oczekuje się, że wynikowa wizualizacja będzie bardziej zrozumiała i atrakcyjna wizualnie.

In [ ]:
# plot of average monthly volume
sp500_volume = sp500['volume'].copy()
sp500_volume = sp500_volume.resample('1M').mean()
volume_plot_avg_m= sp500_volume.plot.line(y="volume", use_index = True)
volume_plot_avg_m.set_xlabel("Date")
volume_plot_avg_m.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: '%1.1fB' % (x * 1e-9)))
volume_plot_avg_m.set_ylabel("Volume")
volume_plot_avg_m.set_title('Average monthly volume')

>Wykres jest dużo bardziej czytelny dzięki zwiekszeniu interwału pomiędzy poszczególnymi danymi do miesiąca czasu. Widać na tym wykresie początkowy trend wzrostowy, a poźniej ustabilizowanie ilości wolumenu w obrocie. Są dwa widoczne okresy większych obrotów w okolicy kryzysu z 2008r. oraz początku pandemii Covid-19.

In [ ]:
# seasonability
sp500_copy = sp500.copy()
sp500_copy['year'] = sp500_copy.index.year
sp500_copy['month'] = sp500_copy.index.month
print(sp500_copy['year'].unique())
sp500_copy

In [ ]:
# seasonability of price column
column = "close"
# create a figure with one plot
fig, ax = plt.subplots(figsize=(15, 6))

# plot the seasonal line plot
sns.lineplot(x=sp500_copy['month'], y=sp500_copy[column], hue = sp500_copy['year'])

# format the plot
ax.set_title('Seasonal plot of Price', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
ax.set_xlabel('Month', fontsize = 16, fontdict=dict(weight='bold'))
ax.set_ylabel('Price', fontsize = 16, fontdict=dict(weight='bold'))
ax.legend(labels = [str(2004+i) for i in range(20)], bbox_to_anchor=(1.1, 1.05))

# create a figure with two plots
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))

# plot the year-wise box plot
sns.boxplot(x=sp500_copy['year'], y=sp500_copy[column], ax=ax[0])

# format the year-wise box plot
ax[0].set_title('Year-wise Box Plot\n(The Trend)', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
ax[0].set_xlabel('Year', fontsize = 16, fontdict=dict(weight='bold'))
ax[0].set_ylabel('Price', fontsize = 16, fontdict=dict(weight='bold'))

# plot the month-wise box plot
sns.boxplot(x=sp500_copy['month'], y=sp500_copy[column], ax=ax[1])

# format the month-wise box plot
ax[1].set_title('Month-wise Box Plot\n(The Seasonality)', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
ax[1].set_xlabel('Month', fontsize = 16, fontdict=dict(weight='bold'))
ax[1].set_ylabel('Price', fontsize = 16, fontdict=dict(weight='bold'))

# rotate the x-axis labels for better readability
fig.autofmt_xdate()

**Seasonal plot of Price**

>Dla poszczególnych lat nie widać wyraźniej sezonowości. Chociaż można zauważyć większą zmienność notowań dla ostatnich lat.

**Year wise Box Plot**

>Na tym wykresie widać ogólny trend wzrostowy. Nie jest widoczna sezonowść. Natomiast można zauważyć znaczne wartości odstające dla 2008r. oraz ostatnich lat. To sugeruje większą zmienność w tych okresach. Dla reszty notowań wartości sa dosyć mocno ograniczone w wąskim zakresie.

**Month wise Box Plot**

>Nie są widoczne dużej rozbieżności. Można jednak zauważyć, że mediana dla miesięcy luty oraz kwiecień jest troszkę niższa od pozostałych miesięcy. 

In [ ]:
# seasonability of volume column
column = "volume"
# create a figure with one plot
fig, ax = plt.subplots(figsize=(15, 6))

# plot the seasonal line plot
sns.lineplot(x=sp500_copy['month'], y=sp500_copy[column], hue = sp500_copy['year'])

# format the plot
ax.set_title('Seasonal plot of Volume', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
ax.set_xlabel('Month', fontsize = 16, fontdict=dict(weight='bold'))
ax.set_ylabel('Volume', fontsize = 16, fontdict=dict(weight='bold'))
ax.legend(labels = [str(2004+i) for i in range(20)], bbox_to_anchor=(1.1, 1.05))

# create a figure with two plots
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))

# plot the year-wise box plot
sns.boxplot(x=sp500_copy['year'], y=sp500_copy[column], ax=ax[0])

# format the year-wise box plot
ax[0].set_title('Year-wise Box Plot\n(The Trend)', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
ax[0].set_xlabel('Year', fontsize = 16, fontdict=dict(weight='bold'))
ax[0].set_ylabel('Volume', fontsize = 16, fontdict=dict(weight='bold'))

# plot the month-wise box plot
sns.boxplot(x=sp500_copy['month'], y=sp500_copy[column], ax=ax[1])

# format the month-wise box plot
ax[1].set_title('Month-wise Box Plot\n(The Seasonality)', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
ax[1].set_xlabel('Month', fontsize = 16, fontdict=dict(weight='bold'))
ax[1].set_ylabel('Volume', fontsize = 16, fontdict=dict(weight='bold'))

# rotate the x-axis labels for better readability
fig.autofmt_xdate()

**Seasonal plot of Volume**

>Dla poszczególnych lat nie widać wyraźniej sezonowości. Można zauważyć mniejszą lość wolumenu dla miesięcy letnich czyli lipca i sierpnia.

**Year wise Box Plot**

>Na tym wykresie jest widoczny początkowy z największą medianą dla roku 2009. Nastepnie ilość wolumenu w obiegu spada i stabilizuje w ostatnich latach. Największa zmienność jest dla lat związanych z większymi spadkami notowań. To zapewne wynika z większej dynamiki cen oraz nerwowości inwestorów. Znaczne wartości odstające pojawiają się od roku 2007 wskazują większą zmienność wolumenu, która stale się utrzymuje.

**Month wise Box Plot**

>Nie są widoczne dużej rozbieżności. Nie można stwerdzić isteniejącej sezonowości 

In [ ]:
# aggregating sp500 stock quotes to a monthly scaled index
sp500_month_agg = sp500.resample('1M').mean()


# # Create the seasonal subseries plots
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(16, 10))

month_plot(sp500_month_agg['close'], ax=ax[0]);
ax[0].set_ylabel('Close Price', fontsize = 16, fontdict=dict(weight='bold'))

month_plot(sp500_month_agg['volume'], ax=ax[1]);
ax[1].set_ylabel('Volume', fontsize = 16, fontdict=dict(weight='bold'))

>Wykresy te aggreguja wartości ze wszsytkich lat dla poszczególnych miesięcy. grupowania.  Dzięki temu można potencjalnie zauważyć wyraźniejszy obraz potencjalnej miesięcznej sezonowości.
>
>Staje się oczywiste, że nie ma sezonowości dla notowań. Średnia miesięcznych cen otwarcia i zamknięcia S&P 500 pozostaje stosunkowo stała.
>
>Dla wolumenu jest widoczna większa zmienność, ale nie jest duża. Średnia miesięcznych ilości obrotów na indeksie S&P 500 różni się nieznacznie dla poszczególnych miesięcy. Widoczne sa mniejsze wartośći dla miesięcy letnich.

**<font color="#5963ab" size="4">nyt_hl</font>**

>New York Times jest jednym z najbardziej wpływowych i rozpoznawalnych dzienników w Stanach Zjednoczonych i na całym świecie.Jest to amerykańska gazeta z siedzibą w Nowym Jorku, znana ze wszechstronnego relacjonowania wiadomości krajowych i międzynarodowych, polityki, biznesu, kultury i wielu innych tematów. Dlatego analiza nagłówków może dawać wskazówki o sytuacji geopolitycznej i ekonomicznej na świecie, co może uda się z powodzeniem wykorzystać do predykcji notowań.

In [ ]:
# check for missing values in each column
missing_columns = nyt_hl.isnull().sum()
pprint("Missing values in column")
pprint(missing_columns)
print()
# print all problematic rows
null_mask = nyt_hl.isnull().any(axis=1)
null_rows = nyt_hl[null_mask]
pprint("Null rows")
pprint(null_rows)
print()
# delete null rows
nyt_hl = nyt_hl.dropna()

In [ ]:
# group all articles for dates to lists
nyt_hl= nyt_hl.groupby(nyt_hl.index).agg({'headline': list})
nyt_hl

In [ ]:
# print all nyt_hl data info
pprint(nyt_hl.info())

In [ ]:
# sum articles for all dates
nyt_hl['art_amount'] = nyt_hl['headline'].str.len()
nyt_hl

In [ ]:
# plot of average monthly amount of published articles 
nyt_hl_sum = nyt_hl['art_amount'].copy()
nyt_hl_sum = nyt_hl_sum.resample('1M').mean()
art_avg_m= nyt_hl_sum.plot.line( use_index = True)
art_avg_m.set_xlabel("Date")
art_avg_m.set_ylabel("Articles")
art_avg_m.set_title('Average monthly published articles')

> Z powyższego wykresu widać niestety duży trend spadkowy w ilości publikowanych artykułów przez NYT. Sa też dwa okresy zwiększonej ilości publikacji. Niestety nie jest to dobrą waidomościa dla wyznacznia predykcji inwestowania. Nierównomierne rozłożenie danych w czasie czyli ich niezbalansowanie to istotny problem. Ma on wpływ na skuteczność modeli uczenia maszynowego i może prowadzić do niepożądanych efektów. 

In [ ]:
# count words for all dates
def count_words(text_list):
    sum = 0
    for text in text_list:
        words = text.split() 
        sum+= len(words)
    return sum

nyt_hl['word_count'] = nyt_hl['headline'].apply(count_words)
nyt_hl

In [ ]:
# plot of average monthly amount of published articles 
nyt_hl_sum = nyt_hl['word_count'].copy()
nyt_hl_sum = nyt_hl_sum.resample('1M').mean()
art_avg_m= nyt_hl_sum.plot.line( use_index = True)
art_avg_m.set_xlabel("Date")
art_avg_m.set_ylabel("Words")
art_avg_m.set_title('Average monthly words in articles')

> Powyższy wykres pokrywa się z poprzednim wykresem. Widać dużo większą ilość słów dla lat z pierwszych lat zebranych danych. 

In [ ]:
# seasonability
nyt_hl_copy = nyt_hl.copy()
nyt_hl_copy['year'] = nyt_hl_copy.index.year
nyt_hl_copy['month'] = nyt_hl_copy.index.month
print(nyt_hl_copy['year'].unique())
nyt_hl_copy

In [ ]:
# seasonability of art_amount column
column = "art_amount"
# create a figure with one plot
fig, ax = plt.subplots(figsize=(15, 6))

# plot the seasonal line plot
sns.lineplot(x=nyt_hl_copy['month'], y=nyt_hl_copy[column], hue = nyt_hl_copy['year'])

# format the plot
ax.set_title('Seasonal plot of articles', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
ax.set_xlabel('Month', fontsize = 16, fontdict=dict(weight='bold'))
ax.set_ylabel('Article', fontsize = 16, fontdict=dict(weight='bold'))
ax.legend(labels = [str(2004+i) for i in range(20)], bbox_to_anchor=(1.1, 1.05))

# create a figure with two plots
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))

# plot the year-wise box plot
sns.boxplot(x=nyt_hl_copy['year'], y=nyt_hl_copy[column], ax=ax[0])

# format the year-wise box plot
ax[0].set_title('Year-wise Box Plot\n(The Trend)', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
ax[0].set_xlabel('Year', fontsize = 16, fontdict=dict(weight='bold'))
ax[0].set_ylabel('Article', fontsize = 16, fontdict=dict(weight='bold'))

# plot the month-wise box plot
sns.boxplot(x=nyt_hl_copy['month'], y=nyt_hl_copy[column], ax=ax[1])

# format the month-wise box plot
ax[1].set_title('Month-wise Box Plot\n(The Seasonality)', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
ax[1].set_xlabel('Month', fontsize = 16, fontdict=dict(weight='bold'))
ax[1].set_ylabel('Article', fontsize = 16, fontdict=dict(weight='bold'))

# rotate the x-axis labels for better readability
fig.autofmt_xdate()

**Seasonal plot of Articles**

>Dla poszczególnych lat nie widać wyraźniej sezonowości. Można zauważyć inne wartości dla poszczegółnych lat oraz większą zmienność dla lat z dużą ilością artykułów

**Year wise Box Plot**

>Na tym wykresie jest widoczny początkowy wzrost z największą medianą dla lat 2006 i 20009. Nastepnie ilość artykułów. Największa zmienność jest dla lat z największa ilością notowań. Znaczne wartości odstające pojawiają się  różnych okresach. W ostatnich latach widać dni w których nadzywczajnie było dużo publikacji co jest zastanwawiające jak rzetelne są te dane pobrane z API NYT.

**Month wise Box Plot**

>Nie są widoczne dużej rozbieżności. Nie można stwerdzić isteniejącej sezonowości 

In [ ]:
# aggregating nyt_hl to a monthly scaled index
nyt_hl_copy = nyt_hl.copy()
del nyt_hl_copy['headline']

nyt_hl_month_agg = nyt_hl_copy.resample('1M').mean()

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(16, 10))

month_plot(nyt_hl_month_agg['art_amount'], ax=ax[0]);
ax[0].set_ylabel('Articles', fontsize = 16, fontdict=dict(weight='bold'))

month_plot(nyt_hl_month_agg['word_count'], ax=ax[1]);
ax[1].set_ylabel('Words', fontsize = 16, fontdict=dict(weight='bold'))

>Nie widać dużej zmienności dla poszczególnych miesięcy dla tych danych. Powoduje to, że nie ma sezonowości w ilości publikacji.

**<font color="#5963ab" size="4">macro_data - sp500_rates</font>**

>Wskaźniki dla akcji, indeksów to liczby lub metryki, które pomagają inwestorom, analitykom finansowym i traderom ocenić wydajność i zdrowie rynku giełdowego, konkretnych akcji lub instrumentów finansowych. Istnieje wiele różnych wskaźników giełdowych, z których każdy koncentruje się na innych aspektach rynku.

In [ ]:
# check for missing values in each column
missing_columns = sp500_rates.isnull().sum()
pprint("Missing values in column")
pprint(missing_columns)

# print all sp500_rates data info
print()
pprint(sp500_rates.info())

>**P/E Ratio** - (kolumna - 'pe')
>
>P/E (Price-to-Earnings ratio) to stosunek ceny akcji do zysku na akcje. Pomaga ocenić, czy akcje są przecenione (niski P/E) lub nadwycenione (wysoki P/E).

In [ ]:
# plot P/E
sp500_pe_plot = sp500_rates['pe'].plot.line( use_index = True)
sp500_pe_plot.set_ylabel("P/E")
sp500_pe_plot.set_xlabel("Date")
sp500_pe_plot.set_title('P/E Ratio - S&P 500')

In [ ]:
# # plot P/E, set_ylim
sp500_pe_plot = sp500_rates['pe'].plot.line( use_index = True)
sp500_pe_plot.set_ylabel("P/E")
sp500_pe_plot.set_xlabel("Date")
sp500_pe_plot.set_title('P/E Ratio - S&P 500')
sp500_pe_plot.set_ylim(10, 50)

> Od razu rzcua się w oczy na wykresie Cean/Zysk duży pik w 2009 roku. Ten wystrzał w górę jest związaney z osiąganiem niskich zysków przez firmy spowodowany kryzsem z 2008r. Dodatkowo 2009r sytuacja się stabilizowała i wyceny spółek poszły do góry. Co spowodowało, aż taką wielkość tego współczynnika.  Wysokie wskaźniki P/E obserwowane w 2009 roku były odzwierciedleniem skrajnych warunków rynkowych w tym okresie. W miarę jak gospodarka wracała do wzrostu, a zyski spółek się poprawiały, wskaźniki P/E stopniowo się normalizowały.
>
>W okresie 2020 do 2022 widać dużą zmienność. Wskażnik w tym okresie także osiągał dużę wartości. Co także jest spowowdowane wieloma czynnikami takimi między innymi jak pandemia czy polityka fiskalna i monetrna państw w tym szczególnie USA.

>**CAPE  Ratio** - (kolumna - 'cape')
>
>CAPE (Cyclically Adjusted Price-to-Earnings) to wskaźnik oceny wyceny akcji, który został opracowany przez laureata Nagrody Nobla, Roberta Shillera. Jest to miara, która uwzględnia inflację i cykl gospodarczy przy ocenie, czy rynki akcji są przeceniane lub nadwyceniane. 
>
>Wskaźnik ten porównuje aktualną cenę akcji do średniego zysku firmy na jedną akcję, skorygowanego o inflację, w ciągu ostatnich 10 lat. Główną ideą CAPE jest to, że przy użyciu wieloletniego przeciętnego zysku eliminuje się krótkoterminowe wahania cyklu gospodarczego. Wskaźnik ten pomaga inwestorom oceniać, czy rynki akcji są wyceniane na podstawie długoterminowych fundamentów, czy może występuje przecena lub nadwyżka wyceny.

In [ ]:
# plot CAPE
sp500_pe_plot = sp500_rates['cape'].plot.line( use_index = True)
sp500_pe_plot.set_ylabel("CAPE")
sp500_pe_plot.set_xlabel("Date")
sp500_pe_plot.set_title('CAPE Ratio - S&P 500')

> Nie ma tutaj jak w P/E bardzo gwałtownych zmian wartości. Dzięki wzięciu wyników finansowcyh z ostanich 10 lat wykres jest bardziej odporny na krótkoterminowe wahania notowań. Minimum lokalne i globalne CAPE osiągał w czasach większych spadków indeksu. Obecnie jego wartość wynosi ponad 30 i jest to dosyć znacznie powyżęj średniej wartości.

>**Earning Yield** - (kolumna - 'earn_yield')
>
>Earnings Yield to wskaźnik finansowy używany do oceny rentowności inwestycji w akcje lub spółki. Jest to stosunek zysku netto firmy do ceny jej akcji lub kapitalizacji rynkowej (wartości rynkowej spółki na giełdzie). Earnings Yield mierzy, ile procentowo zysku netto generowanego przez spółkę przypada na jedną jednostkę jej akcji. Wartość tego wskaźnika można interpretować podobnie jak wskaźnik P/E (Price-to-Earnings Ratio), ale w odwrotny sposób. Wskaźnik P/E mierzy, ile razy cena akcji zawiera się w zysku netto, podczas gdy Earnings Yield wyraża zysk netto jako procent ceny akcji.
>
>Earnings Yield może być przydatny w porównywaniu różnych spółek lub rynków giełdowych pod kątem potencjalnej rentowności inwestycji. Wyższy Earnings Yield oznacza, że inwestorzy mogą spodziewać się wyższej rentowności z inwestycji w akcje danej spółki.

In [ ]:
# plot Earning Yield
sp500_pe_plot = sp500_rates['earn_yield'].plot.line( use_index = True)
sp500_pe_plot.set_ylabel("EY")
sp500_pe_plot.set_xlabel("Date")
sp500_pe_plot.set_title('Earning Yield - S&P 500')

> Ten wskażnik jest odwrotnością P/E, więc można wysunać podobne wnioski. Widzimy, że obecnie EY nie jest zbyt wysoko co oznacza, że rentowność nie jest duża w porównaniu do notowań historycznych.

>**PBV** - (kolumna - 'pbv')
>
>Wskaźnik PBV (Price-to-Book Value) jest wskaźnikiem finansowym, który służy do oceny wyceny spółki na podstawie jej wartości księgowej. Jest to stosunek ceny rynkowej akcji spółki do jej wartości księgowej na jedną akcję.
>
>Niższy wskaźnik PBV może sugerować, że akcje są bardziej przystępne cenowo, podczas gdy wyższy wskaźnik może wskazywać na przecenę. Wartości PBV poniżej 1 mogą sugerować, że akcje są wyceniane poniżej wartości księgowej. 

In [ ]:
# plot Earning Yield
sp500_pe_plot = sp500_rates['pbv'].plot.line( use_index = True)
sp500_pe_plot.set_ylabel("PBV")
sp500_pe_plot.set_xlabel("Date")
sp500_pe_plot.set_title('PBV - S&P 500')

>Na wykresie widać, że w ostatnich 20 latach indeks dla 500 najwiekszych spółek USA miał wartość PBV dużo powyżej 1. Nie tak dawno osiągał swoje maksima, osiągając wartość prawie 5. 
>
>Jednak wskaźnik PBV powinien być interpretowany w kontekście zmian w jakie zachodzą w gospodarce. Ponieważ różne branże mogą mieć różne typowe zakresy wartości PBV. Ostatnio coraz większe znaczenia mają firmy z branży technologicznej, a nie przemysłowej, które zazwyczaj posiadają mniej aktywów. Może to powodować większą wartość PBV.

>**P/S Ratio** - (kolumna - 'prices_sales_ratio')
>
>Wskaźnik P/S (Price-to-Sales Ratio), nazywany również wskaźnikiem wyceny na podstawie przychodów, to wskaźnik finansowy wykorzystywany do oceny wyceny spółek. Jest to stosunek ceny akcji danej spółki do jej przychodów ze sprzedaży na jedną akcję lub na jednostkę udziałową.
>
> Niższy wskaźnik P/S może sugerować, że spółka jest bardziej przystępnie wyceniana w porównaniu do swoich przychodów, podczas gdy wyższy wskaźnik P/S może wskazywać na przecenę lub oczekiwania rynku dotyczące przyszłego wzrostu przychodów.
Wskaźnik P/S jest szczególnie przydatny w analizie spółek, które generują duże przychody, ale niekoniecznie osiągają znaczne zyski netto. Jest on również często stosowany w analizie firm technologicznych oraz spółek z sektora wzrostowego, gdzie perspektywa wzrostu przychodów jest kluczowa.


In [ ]:
# plot P/S Ratio
sp500_pe_plot = sp500_rates['price_sales_ratio'].plot.line( use_index = True)
sp500_pe_plot.set_ylabel("P/S")
sp500_pe_plot.set_xlabel("Date")
sp500_pe_plot.set_title('P/S Ratio - S&P 500')

>Obecnie wartość P/S Ratio jest dosyć wysoka. Widać wyraźny trend wzrostowy z maksimum w 2021r. Minimum w 2009r.

>**Dividend Yield** - (kolumna - 'div_yield')
>
>Wskaźnik Dividend Yield (Wydajność Dywidendy) to wskaźnik finansowy, który pomaga inwestorom ocenić stopę zwrotu z dywidend, jaką mogą uzyskać inwestując w akcje danej spółki. Wyraża on procentowy stosunek rocznej dywidendy wypłacanej przez spółkę do jej aktualnej ceny akcji. 
>
>Wskaźnik Dividend Yield informuje inwestorów o tym, ile procentowo dywidendy stanowią w stosunku do aktualnej wartości rynkowej akcji. Wyższy Dividend Yield oznacza, że inwestorzy mogą oczekiwać wyższej stopy zwrotu z dywidend w porównaniu do ceny akcji. Jest to szczególnie atrakcyjne dla inwestorów poszukujących stałego i stabilnego dochodu z inwestycji.
Jednak warto pamiętać, że wyższy Dividend Yield może być spowodowany przez niższą cenę akcji, co może wynikać z różnych czynników, w tym z niedocenienia spółki lub problemów finansowych. Dlatego inwestorzy powinni oceniać Dividend Yield w kontekście innych czynników, takich jak stabilność i wzrost dywidendy, zdolność spółki do generowania dochodu oraz ogólna sytuacja na rynku.

In [ ]:
# plot Dividend Yield
sp500_pe_plot = sp500_rates['div_yield'].plot.line( use_index = True)
sp500_pe_plot.set_ylabel("DY")
sp500_pe_plot.set_xlabel("Date")
sp500_pe_plot.set_title('Dividend Yield - S&P 500')

>Największe wartośći zwrot z dywidendy miał w czasie kryzysu z 2008r. Najniższe wartośći na przełomie 2021/2022r. Obecnie wartość nie jest za wysoka i wynosi około 1.5%.

>**Dividend Growth** - (kolumna - 'div_growth')
>
>Wskaźnik Dividend Growth (Wzrost Dywidendy) to wskaźnik finansowy mierzący tempo wzrostu dywidend wypłacanych przez spółkę, idneks w ciągu określonego okresu czasu. Jest to wskaźnik często analizowany przez inwestorów, zwłaszcza tych, którzy poszukują inwestycji, które regularnie zwiększają swoje wypłaty dywidendowe. Wzrost dywidend dla spółki jest uważany za wskaźnik zdrowego i stabilnego zarządzania finansami spółki oraz może sugerować, że spółka ma perspektywy wzrostu.
>
>Wskaźnik Dividend Growth wyraża względny wzrost dywidendy jako procent. Wskaźnik Dividend Growth jest ważnym narzędziem dla inwestorów, ponieważ może świadczyć o zdolności spółki do generowania rosnącego dochodu dla swoich akcjonariuszy. Jednak warto zwrócić uwagę, że wzrost dywidendy może być wynikiem wielu czynników, w tym poprawy wyników finansowych, zysków spółki, a także polityki dywidendowej zarządu. Jednak w tym indeksie mamy 500 spółek więc pojedyncze specyficzne zachowania spółek nie mają tak dużej wagi i widzimy jak ogólnie cały rynek prowadzi politykę dywidend.
>
>Wskaźnik tutaj jest obliczany kwartalnie. Czy zmiana procentowa jest oblicznana na podstawie wysokości dywidend z dwóch ostatnich kwartałów. Dzięki czemu będzię można zauważyć szybciej zmiany na rynku, ale może być więcej szumu, nagłych zmian.

In [ ]:
# plot Dividend Growth
sp500_pe_plot = sp500_rates['div_growth'].plot.line( use_index = True)
sp500_pe_plot.set_ylabel("DG")
sp500_pe_plot.set_xlabel("Date")
sp500_pe_plot.set_title('Dividend Growth - S&P 500')

Na wykresie widać duży spadek dywidend dla okresów związanych z pogorszeniem sytuacji gospodarczej. Co wiąże się z mniejszymi zyskami w tym okresie. Obecnie wartość Dividend Growth wynosi 7,33%.

**<font color="#5963ab" size="4">macro_data - usa_eco_rates</font>**

>Wskaźniki makroekonomiczne to liczby i statystyki używane do pomiaru i analizy zdrowia i wydajności gospodarki na poziomie makroekonomicznym, czyli ogólnym. Te wskaźniki dostarczają informacji na temat kluczowych aspektów gospodarki danego kraju lub regionu i pozwalają na ocenę jej kondycji, trendów oraz zdolności do wzrostu. Wskaźniki makroekonomiczne są szeroko stosowane przez rządy, banki centralne, inwestorów i analityków do podejmowania decyzji i oceny polityki gospodarczej. 

In [ ]:
# check for missing values in each column
missing_columns = usa_eco_rates.isnull().sum()
pprint("Missing values in column")
pprint(missing_columns)

# print all usa_eco_rates data info
print()
pprint(usa_eco_rates.info())

>**Gross domestic product (GDP)** - (kolumna - 'gdp')
>
>GDP czyli PKB, Produkt Krajowy Brutto, to kluczowy wskaźnik makroekonomiczny, który mierzy ogólną wartość wszystkich dóbr i usług wytworzonych w danym kraju w określonym okresie czasu, zwykle w ciągu roku. PKB jest używany do pomiaru wielkości i wydajności gospodarki danego kraju i stanowi ważny wskaźnik do analizy i oceny jej zdrowia ekonomicznego.

In [ ]:
# plot Gross domestic product
usa_eco_rates_plot = usa_eco_rates['gdp'].plot.line( use_index = True)
usa_eco_rates_plot.set_ylabel("GDP")
usa_eco_rates_plot.set_xlabel("Date")
usa_eco_rates_plot.set_title('Gross domestic product')

>Widać duży trend wzrostowy jeśli chodzi o PKB USA. Są zauważalne dwa spadki PKB związne z kryzesem w 2008r. oraz z pandemią Covid-19. Jednak te spadki zostały z dużą nawiązką szybko odrobione.

>**GDP Percent Change** - (kolumna - 'gdp_percent_change')
>
>Zmiana procentowa PKB, zwana także wzrostem lub spadkiem procentowym PKB, to wskaźnik używany do określenia zmiany ogólnej wartości Produktu Krajowego Brutto (PKB) w określonym okresie czasu, wyrażonej jako procent. Ten wskaźnik jest ważnym narzędziem do monitorowania tempa wzrostu lub spadku gospodarki. Pozytywna zmiana procentowa oznacza wzrost gospodarczy, podczas gdy ujemna zmiana procentowa oznacza spadek gospodarczy.
>
>Zmiana procentowa PKB jest kluczowym wskaźnikiem do oceny tempa wzrostu gospodarczego lub recesji w danym kraju lub regionie. Jest często używana przez analityków, inwestorów i ekonomistów do analizy kondycji gospodarczej i prognozowania przyszłych trendów gospodarczych.
>
>Wskaźnik tutaj jest obliczany kwartalnie. Czy zmiana procentowa jest oblicznana na podstawie wysokości PKB z dwóch ostatnich kwartałów. Dzięki czemu będzię można zauważyć szybciej zmiany na rynku, ale podobnie jak dla Dividend Growth może być więcej szumu i nagłych zmian.

In [ ]:
# plot GDP quartely percent change
usa_eco_rates_plot = usa_eco_rates['gdp_percent_change'].plot.line( use_index = True)
usa_eco_rates_plot.set_ylabel("GDP_PCH")
usa_eco_rates_plot.set_xlabel("Date")
usa_eco_rates_plot.set_title('GDP quartely percent change')

>Wykres prezentuje dość stabilny kilkuprocentowy kwartalny wzrost PKB gospodarki USA. W czasie kryzysu z 2008r. widoczna jest recejsa gospodarki. W czasie pandemii i spowodowanej przez nie lockdownu widoczna jest bardzo duża recejsa, ale jest ona krótkotrwała i szybko odbudowana.

>**Unemployment rate** - (kolumna - 'unemploy_rate')
>
>Krótka stopa bezrobocia, często nazywana również "stopą bezrobocia" lub "stopą bezrobocia ogólnokrajową", to wskaźnik, który mierzy odsetek ludności aktywnej zawodowo, która jest bez pracy i aktywnie jej poszukuje w danym kraju w określonym okresie czasu. Jest to jeden z kluczowych wskaźników rynku pracy i jest często używany do oceny zdrowia gospodarki oraz efektywności polityki rynku pracy.
>
> Im niższa jest stopa bezrobocia, tym bardziej korzystna jest sytuacja na rynku pracy, ponieważ oznacza to, że większość osób jest zatrudniona. Natomiast wyższa stopa bezrobocia może świadczyć o trudnościach na rynku pracy i słabszym stanie gospodarki.

In [ ]:
# plot Unemployment rate
usa_eco_rates_plot = usa_eco_rates['unemploy_rate'].plot.line( use_index = True)
usa_eco_rates_plot.set_ylabel("Unemployment rate")
usa_eco_rates_plot.set_xlabel("Date")
usa_eco_rates_plot.set_title('Unemployment rate')

>Wykres stopy bezrobocia w USA jest dość zmienny i prezentuje dwa wzrosty spwodowane recesją gospodarki. Widać, że w 2008 roku bezrobocie wzrastało nawet po kryzysie i miało dużą bezwładność. Zajęło kilka lat do powrotu do 5% bezrobocia po kryzysie. Natomiast dla pandemii, bardzo dynamiczny wzrost bezrobocia, takżę ma swoją bezwładność, ale został w miarę szybko zniwelowany. Obecnie amerykańska ma bardzo niskie bezrobocie wynoszące koło 3%.

>**Purchasing Managers' Index** - (kolumna - 'pmi_index')
>
>PMI (Purchasing Managers' Index) to indeks zakupów, który mierzy aktywność w sektorze produkcyjnym i usługowym gospodarki. Jest to ważny wskaźnik ekonomiczny używany do monitorowania wydajności gospodarki oraz oceny kierunku, w jakim zmierza. Indeks PMI jest szeroko stosowany na całym świecie i dostarcza informacji na temat kondycji gospodarki oraz tendencji koniunkturalnych.
>
>Ogólna idea indeksu PMI polega na zbieraniu informacji od menedżerów odpowiedzialnych za zakupy (purchasing managers) w firmach. Menedżerowie ci są pytani o różne aspekty działalności swoich firm, takie jak liczba nowych zamówień, produkcja, zatrudnienie, poziom cen surowców, itp. Odpowiedzi te są następnie agregowane i przeliczane na wskaźnik PMI.
>Indeks PMI jest publikowany regularnie, zwykle co miesiąc, i stanowi ważne narzędzie analizy rynku. Jest używany w wielu krajach na całym świecie i pomaga w zrozumieniu zmian w aktywności gospodarczej oraz w podejmowaniu decyzji zarówno na rynkach finansowych, jak i w biznesie.

In [ ]:
# plot PMI Index
usa_eco_rates_plot = usa_eco_rates['pmi_index'].plot.line( use_index = True)
usa_eco_rates_plot.set_ylabel("PMI Index")
usa_eco_rates_plot.set_xlabel("Date")
usa_eco_rates_plot.set_title('PMI Index')

Duża zmienność jest widoczna na tym wykresie. Najniższe wartośći notowane są w czasach recesji gospodarki USA. Obecnie wskaźnik w stosunku do ostatnich 20 lat jest nieznacznie poniżej 50 co jest niską wartością.

>**Consumer Price Index for All Urban Consumers** - (kolumna - 'cpiaucsl_pch')
>
>CPIAUCSL to skrót od "Consumer Price Index for All Urban Consumers". Jest to indeks cen konsumenckich obliczany w Stanach Zjednoczonych przez Biuro Statystyki Pracy (Bureau of Labor Statistics - BLS). Indeks CPIAUCSL jest używany do pomiaru zmian cen towarów i usług zakupowanych przez gospodarstwa domowe w obszarach miejskich w USA. Jest to jeden z głównych wskaźników inflacji i pomaga w monitorowaniu wzrostu cen na rynku konsumenckim.
>
>W kontekście indeksu CPI-U, "percent change" (zmiana procentowa) odnosi się do procentowej zmiany wartości indeksu w określonym okresie czasu. W tych danych wskaźnik jest liczony w każdy miesiąć do okresu wcześniejszego. Indeks PMI jest publikowany regularnie, zwykle co miesiąc, i stanowi ważne narzędzie analizy rynku. Jest to kluczowy wskaźnik ekonomiczny używany przez ekonomistów, banki centralne i rządy do analizy zdrowia gospodarki i podejmowania decyzji politycznych.

In [ ]:
# plot Consumer Price Index for All Urban Consumers
usa_eco_rates_plot = usa_eco_rates['cpiaucsl_pch'].plot.line( use_index = True)
usa_eco_rates_plot.set_ylabel("CPI PCH")
usa_eco_rates_plot.set_xlabel("Date")
usa_eco_rates_plot.set_title('CPI monthly percent change')

>Z powodu częstego mierzenia tego indeksu widzimy dużą zmienność. W czasach recesji mamy deflację, szczególnie widoczną dla okresy z kryzysu 2008r. Dla lat wzrostu gospodarczego ideks jest nieznacznie powyżej zera. W 2021 roku widać duzy wzrost inflacji, która obecnie wydaje się być ustablilizowana i wynosi 0,17% w stosunku do porzedniego miesiąca. 

>**Effective Federal Funds Rate (EFFR)** - (kolumna - 'fed_effective_rate')
>
>Effective Federal Funds Rate" (EFFR), znana także jako "federal funds effective rate", to jedna z kluczowych stóp procentowych w Stanach Zjednoczonych. Jest to stopa procentowa, po której banki komercyjne udzielają sobie pożyczek na rynku międzybankowym, zwanych "federal funds market". EFFR jest jednym z głównych narzędzi, które Federal Reserve (Fed), amerykański bank centralny, używa do kontrolowania polityki monetarnej.
>
>EFFR jest kluczowym narzędziem, które pozwala Fed regulować poziom aktywności gospodarczej, kontrolować inflację i stabilizować rynki finansowe. Jest to ważny wskaźnik, który jest śledzony przez ekonomistów, inwestorów i instytucje finansowe, ponieważ jego zmiany mają wpływ na szeroki zakres aspektów gospodarki i rynków finansowych.

In [ ]:
# plot Effective Federal Funds Rate
usa_eco_rates_plot = usa_eco_rates['fed_effective_rate'].plot.line( use_index = True)
usa_eco_rates_plot.set_ylabel("EFFR")
usa_eco_rates_plot.set_xlabel("Date")
usa_eco_rates_plot.set_title('Effective Federal Funds Rate')

>Na wykresie widać dwa duże wzrosty efektywnych stóp procentowych do wartości powyżej 5%. Obecnie na takim poziomie są stopy procentowe, co jest spowodowane niedawną wysoką inflacją w USA. Można zauważyć, że przez długi okres stopy procentowe wynosiło koło 0%.

>**Effective Federal Funds Rate Percent Change** - (kolumna - 'fed_effective_rate_pch')
>
>Procentowa zmiana stopy "Effective Federal Funds Rate" (EFFR) odnosi się do procentowej zmiany tej stopy procentowej w określonym okresie czasu. Procentowa zmiana w EFFR jest obliczana przez porównanie obecnej stopy do wcześniejszej stopy, zazwyczaj z miesiąca, kwartału lub roku wcześniej. Pomaga to ocenić, o ile procent stopa procentowa wzrosła lub spadła w tym okresie czasu. W tych dancyh jest obliczana co miesiąc.
>
>Procentowa zmiana stopy "Effective Federal Funds Rate" jest istotnym wskaźnikiem służącym do analizowania tendencji w krótkoterminowych stopach procentowych w Stanach Zjednoczonych. Jest śledzona uważnie przez ekonomistów, decydentów politycznych i inwestorów, ponieważ może dostarczać informacji na temat stanowiska Federalnej Rezerwy w kwestii polityki pieniężnej oraz jej działań mających na celu kontrolowanie inflacji i wspieranie wzrostu gospodarczego.

In [ ]:
# plot Effective Federal Funds Rate Monthly Percent Change
usa_eco_rates_plot = usa_eco_rates['fed_effect_rate_pch'].plot.line( use_index = True)
usa_eco_rates_plot.set_ylabel("EFFR PCH")
usa_eco_rates_plot.set_xlabel("Date")
usa_eco_rates_plot.set_title('Effective Federal Funds Rate Percent Change')

>Na wykresie widać duże okresy stablizacji efektywnych stóp procentowych. Jednak są widoczne trzy okresy - kryzys 2008, pandemia 2020, inflacja 2021, w których nastąpiły duże i częste zmiany stóp procentowych.

>**USA 10-Year Bond Yield** - (kolumna - '10usy_bond_yield')
>
>"USA 10-year bond yield" oznacza rentowność lub stopę procentową na 10-letnich obligacjach skarbowych Stanów Zjednoczonych. Reprezentuje to zannualizowany zwrot, jaki inwestor może oczekiwać, trzymając 10-letnią obligację rządową Stanów Zjednoczonych do daty wykupu.
>
>Rentowność 10-letnich obligacji skarbowych USA to kluczowy wskaźnik finansowy, który odzwierciedla stopę procentową na 10-letnich obligacjach rządowych Stanów Zjednoczonych. Gra on istotną rolę na rynkach finansowych i pełni funkcję wskaźnika ekonomicznego, który odzwierciedla warunki gospodarcze i nastroje inwestorów.

In [ ]:
# plot USA 10-Year Bond Yield
usa_eco_rates_plot = usa_eco_rates['10usy_bond_yield'].plot.line( use_index = True)
usa_eco_rates_plot.set_ylabel("10Y Bond Yield")
usa_eco_rates_plot.set_xlabel("Date")
usa_eco_rates_plot.set_title('USA 10-Year Bond Yield')

Ogólnie na wykresie jest widoczny trend spadkowy prezz większość okresu czasu. Od wartośći powyżej 4% rentowność 10 letnich oblifgacji spadła prawie do zera w 2020r.. W ostatnich 2 latach widać odwrócenie trendu i duży szybki wzrost do wartości powyżej 4%.

>**Consumer Sentiment** - (kolumna - 'cons_sent')
>
>Jeden z popularnych wskaźników mierzących nastroje konsumentów w Stanach Zjednoczonych. Mierzony przez "University of Michigan Consumer Sentiment Index" (Indeks Nastrojów Konsumentów Uniwersytetu Michigan)."Consumer sentiment" odnosi się do ogólnego nastroju i postaw konsumentów wobec gospodarki, finansów osobistych oraz przyszłości. Jest to wskaźnik mierzący poziom pewności i optymizmu konsumentów, który może wpływać na ich decyzje zakupowe i wydatki.
>
>Indeks Nastrojów Konsumentów jest używany jako wskaźnik do analizy klimatu konsumenckiego i może dostarczać wskazówek dotyczących przyszłego zachowania rynku. Jest to również ważny element analizy ekonomicznej, który pomaga monitorować zmiany nastrojów konsumentów w odpowiedzi na zmieniające się warunki gospodarcze i wydarzenia na rynku.

In [ ]:
# plot Consumer Sentiment
usa_eco_rates_plot = usa_eco_rates['cons_sent'].plot.line( use_index = True)
usa_eco_rates_plot.set_ylabel("Consumer Sentiment")
usa_eco_rates_plot.set_xlabel("Date")
usa_eco_rates_plot.set_title('Consumer Sentiment')

Im niższa wartość tym gorsze nastroje konsumentów. W recesjach gospodarki widoczne są najniższe wartosci tego indeksu. Obecnie indeks nastoje poprawiają się po pandemii Covid-19, ale patrząc na cały wykres dalej nie są na wysokim pozomie.

>**Consumer Sentiment Percent Change** - (kolumna - 'cons_sent_percent_change')
>
>J"Consumer sentiment percent change" dla Stanów Zjednoczonych odnosi się do procentowej zmiany wskaźnika nastrojów konsumentów (consumer sentiment) w danym okresie w porównaniu do poprzedniego okresu. Jest to miara, która informuje o tym, o ile procent wskaźnik nastrojów konsumentów wzrósł lub spadł w określonym czasie.
>
>Ten wskaźnik jest istotny, ponieważ pomaga analizować, czy nastroje konsumentów stają się bardziej optymistyczne (jeśli wartość jest dodatnia) lub bardziej pesymistyczne (jeśli wartość jest ujemna) w stosunku do wcześniejszego okresu. To może dostarczać informacji na temat zmian w postrzeganiu gospodarki i wpływać na decyzje konsumentów dotyczące zakupów i wydatków.
>
>Zmiany w nastrojach konsumentów są istotne dla ekonomistów, inwestorów i polityków, ponieważ mogą wpłynąć na zachowanie gospodarki, rynek pracy i inne aspekty ekonomiczne. Dlatego "Consumer sentiment percent change" jest śledzony i analizowany jako część badania nastrojów konsumentów.

In [ ]:
# plot Consumer Sentiment Monthly Percent Change
usa_eco_rates_plot = usa_eco_rates['cons_sent_percent_change'].plot.line( use_index = True)
usa_eco_rates_plot.set_ylabel("Consumer Sentiment PCH")
usa_eco_rates_plot.set_xlabel("Date")
usa_eco_rates_plot.set_title('Consumer Sentiment Monthly Percent Change')

Widać dużą zmienność tego wykresu. Wykres jest bardzo trudny do odczytu. Ilośc zmian powoduje, że trudno jest wyczytać coś z tego wykresu. Możliwe, że zwiększenie okresu porównywania wskażnika do roku spowodowało by większą czytelność i możliowść wysnucia ciekawych wniosków.

**<font color="#5963ab" size="4">goog_trend</font>**

>Google Trends to narzędzie dostarczane przez Google, które umożliwia analizowanie popularności i wyszukiwań określonych fraz lub słów kluczowych w wyszukiwarce Google w określonym czasie i w określonych regionach geograficznych. To narzędzie pozwala zrozumieć, jakie tematy są popularne w danym momencie oraz jak zmienia się ich popularność w czasie.
>
>Google Trends jest używane w różnych celach, takich jak analiza rynku, badanie trendów marketingowych, badanie zainteresowań społeczności online, śledzenie popularności produktów i wiele innych. Jest to przydatne narzędzie zarówno dla profesjonalistów w dziedzinie marketingu, jak i dla osób śledzących trendy w danym obszarze tematycznym.


In [ ]:
# check for missing values in each column
missing_columns = goog_trend.isnull().sum()
pprint("Missing values in column")
pprint(missing_columns)
# print all goog_trend data info
print()
pprint(goog_trend.info())


In [ ]:
# plot trend
goog_trend_plot = goog_trend.plot.line( use_index = True)
goog_trend_plot.set_ylabel("Trend")
goog_trend_plot.set_title('S&P 500 trend')

>Z wykresu widać początkowo stabilną popularność wyszuliwań indeksu, ale od około roku 2015 następuje większa popularność. Szczyt równy 100 (trend jest mierzony w zakresie od 0 do 100) przypada w okresie wybuchu pandemii Covid-19.

In [ ]:
# print date with max popularity
pprint(goog_trend.idxmax().max())
pprint(goog_trend.max().max())

In [ ]:
# seasonability
goog_trend_copy = goog_trend.copy()
goog_trend_copy['year'] = goog_trend_copy.index.year
goog_trend_copy['month'] = goog_trend_copy.index.month
print(goog_trend_copy['year'].unique())
goog_trend_copy.columns

In [ ]:
# seasonability of trend column
column = "month_trend"
# create a figure with one plot
fig, ax = plt.subplots(figsize=(15, 6))

# plot the seasonal line plot
sns.lineplot(x=goog_trend_copy['month'], y=goog_trend_copy[column], hue = goog_trend_copy['year'])

# format the plot
ax.set_title('Seasonal plot of trend', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
ax.set_xlabel('Month', fontsize = 16, fontdict=dict(weight='bold'))
ax.set_ylabel('Trend', fontsize = 16, fontdict=dict(weight='bold'))
ax.legend(labels = [str(2004+i) for i in range(20)], bbox_to_anchor=(1.1, 1.05))

# create a figure with two plots
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))

# plot the year-wise box plot
sns.boxplot(x=goog_trend_copy['year'], y=goog_trend_copy[column], ax=ax[0])

# format the year-wise box plot
ax[0].set_title('Year-wise Box Plot\n(The Trend)', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
ax[0].set_xlabel('Year', fontsize = 16, fontdict=dict(weight='bold'))
ax[0].set_ylabel('Trend', fontsize = 16, fontdict=dict(weight='bold'))

# plot the month-wise box plot
sns.boxplot(x=goog_trend_copy['month'], y=goog_trend_copy[column], ax=ax[1])

# format the month-wise box plot
ax[1].set_title('Month-wise Box Plot\n(The Seasonality)', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
ax[1].set_xlabel('Month', fontsize = 16, fontdict=dict(weight='bold'))
ax[1].set_ylabel('Trend', fontsize = 16, fontdict=dict(weight='bold'))

# rotate the x-axis labels for better readability
fig.autofmt_xdate()

**Seasonal plot of Volume**

>Dla poszczególnych lat nie widać wyraźnej sezonowości. Widoczny jest duży wzrost popularnoośći frazy "S&P 500" w marcu 2020r.

**Year wise Box Plot**

>Na tym wykresie trend jest prez pierwsze 10 lat i stabilny na równym poziomie. Następnie widać coraz większą popularność wyszukiwań, co może swiadczyć o większej liczbie osób zaintereowanych inwestowaniem. Największą mediana wyniosła dla roku 2022. Największa zmienność jest dla roku 2020 jest to związane z pandemią. Znaczne wartości odstające pojawiają siętakżę w tym roku.

**Month wise Box Plot**

>Nie są widoczne dużej rozbieżności. Nie można stwerdzić isteniejącej sezonowości 

In [ ]:
# aggregating trend to monthly scaled index
fig, ax = plt.subplots()

month_plot(goog_trend['month_trend'], ax=ax);
ax.set_ylabel('Trend', fontsize = 16, fontdict=dict(weight='bold'))

>Nie widać dużej zmienności dla poszczególnych miesięcy dla tych danych. Jedynie marzec ma większą średnią popularnośc. Co jest związane z marcem 2020r. i osiągnieceim największej popularności. Znacznie większej niż dla` innych okresów.

<div id="chap4"><h1 style="color:white;background:#5963ab;border-radius:5px;padding:30px;font-family:'Arial', cursive;font-size:50px;text-align:center">Przetwarzanie wstępne danych</h1></div>

**<font color="#5963ab" size="4">sp500</font>**

In [ ]:
# basic stats about data values
sp500.describe()

In [ ]:
# delete unneeded columns
del sp500['dividends']
del sp500['stock splits']
sp500

In [ ]:
# calculate the correlation matrix
correlation_matrix = sp500.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

> Jak można było się domyśleć korelacja wynosi 1 dla czterechz zmiennych odnoszączych się do ceny. Natomiast pomiędzy notowaniami indeksu a ilością wolumenu jest korelacja niewielka wynosząca 0.19.

In [ ]:
# leave only close price column
del sp500['open']
del sp500['high']
del sp500['low']
sp500

In [ ]:
# make a new columne with daily price percent change
sp500['daily_pch'] = sp500['close'].pct_change() * 100
sp500

In [ ]:
plt.figure(figsize=(20, 14))
sp500['daily_pch'].hist(bins=200)
plt.xlabel('Daily Return in Percent')
plt.ylabel('Counts')
plt.title('Daily Percent Change SP500 - Histogram')

In [ ]:
# calculate the number of daily return above zero, equal to zero, and below zero
above_zero_count = (sp500['daily_pch'] > 0).sum()
equal_to_zero_count = (sp500['daily_pch'] == 0).sum()
below_zero_count = (sp500['daily_pch'] < 0).sum()

# Print the counts
print(f'Number of daily returns above zero: {above_zero_count}')
print(f'Number of daily returns equal to zero: {equal_to_zero_count}')
print(f'Number of daily returns below zero: {below_zero_count}')

In [ ]:
# calculate the percent daily returns above zero, equal to zero, and below zero
num_rows = sp500.shape[0]

# Print the counts
print(f'Percent of daily returns above zero: {above_zero_count/num_rows*100:.2f}%')
print(f'Percent of daily returns equal to zero: {equal_to_zero_count/num_rows*100:.2f}%')
print(f'Percent of daily returns below zero: {below_zero_count/num_rows*100:.2f}%')

In [ ]:
# calculate the percent change between the first and last day for index, total return fow whole period
first_row_value = sp500['close'][0]
last_row_value = sp500['close'][-1]
percent_change = ((last_row_value - first_row_value) / first_row_value) * 100

print(f'Percent change S&P 500 for index: {percent_change:.2f}%')

>Z powyższych danych można zauważyć że większość dni końćzyła się poztywnie dla inwestorów. Około 54.32% notwań zakończyło się na plusie dla indeksu S&P 500. W czasie okresu badań wynoszącego prawie 20 lat indeks wzrósł znacznie ponieważ aż o 306.65%

In [ ]:
# create column "target" for prediction for next day - True if the index increases the next day, or False if it decreases
sp500["tomorrow"] = sp500["close"].shift(-1)
sp500.dropna(subset=['tomorrow'], inplace=True)
sp500["target"] = (sp500["tomorrow"]> sp500["close"]).astype(int)
del sp500['tomorrow']
sp500.head(10)

**<font color="#5963ab" size="4">nyt_hl</font>**

Analiza sentymentunyt_hl

In [ ]:
nyt_hl_copy = nyt_hl.copy()
nyt_hl = nyt_hl_copy

In [ ]:
nyt_hl['headline'][0]

In [ ]:
model_id = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_pipeline = pipeline("sentiment-analysis", model = model_id)

def find_sentiment(text):
    sent = sentiment_pipeline(text)[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

In [ ]:
nyt_hl

In [ ]:
nyt_hl['sentiments'] = np.empty((len(nyt_hl), 0)).tolist()
nyt_hl['total_sent'] = 0
nyt_hl["sentiment"] = 0
nyt_hl["neg_sentiment"] = 0
for idx, headlines in enumerate (nyt_hl['headline'][0:1]): 
    total_sent = 0
    for headline in headlines:
        sent_result = find_sentiment(headline)
        total_sent += sent_result
        nyt_hl['sentiments'][idx].append(sent_result)
    nyt_hl[idx,'total_sent'][idx] = total_sent
    nyt_hl["sentiment"][idx] = mean(nyt_hl['sentiments'][idx])
    nyt_hl["neg_sentiment"][idx] = len([s for s in nyt_hl['sentiments'][idx] if s < 0]) / len(nyt_hl['sentiments'][idx])
    
    #del nyt_hl['sentiments'][idx]

In [ ]:
nyt_hl

In [ ]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = statistics.mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0
    
    del edits[key]["sentiments"]

To raczej powinno być przenieoine do stworzenia modeli bo tutaj sa dame, ale mozne przygotwac te dane by były gtowe do uzycia


In [ ]:
# crete new data frame to NLP process headlines for using further for investing prediction
nlp_data = pd.merge(sp500['target'], nyt_hl['headline'], on='date')
nlp_data

In [ ]:
# calculate the index for the split point
split_point = int(0.8 * len(nlp_data))  # 80% for training, 20% for testing
split_point

In [ ]:
nlp_data.reset_index(drop = True, inplace = True)
# split the data into training and testing sets
train_data = nlp_data.iloc[:split_point]
test_data = nlp_data.iloc[split_point:]
test_data.reset_index(drop = True, inplace = True)

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
# splitting the dataset
y_train = train_data['target']
train = train_data['headline']
y_test = test_data['target']
test = test_data['headline']

In [ ]:
train.info()
train[0]

In [ ]:
train = train.apply(lambda x: '. '.join(x))
test = test.apply(lambda x: '. '.join(x))

In [ ]:
train[0]

In [ ]:
# removing punctuation and special character from the text
train.replace(to_replace='[^a-zA-Z]', value=' ', regex=True, inplace=True)
test.replace(to_replace='[^a-zA-Z]', value=' ', regex=True, inplace=True)

In [ ]:
train[0]

In [ ]:
# converting the entire text to lower case
train = train.str.lower()
test = test.str.lower()

In [ ]:
# creating corpus of train dataset
ps = PorterStemmer()
train_corpus = []

for i in range(0, len(train)):
  
  # tokenizing the news-title by words
  words = train[i].split()

  # removing the stopwords
  words = [word for word in words if word not in set(stopwords.words('english'))]

  # stemming the words
  words = [ps.stem(word) for word in words]

  # joining the stemmed words
  headline = ' '.join(words)

  # building a corpus of news-title
  train_corpus.append(headline)

In [ ]:
train_corpus[0:1]

In [ ]:
# creating corpus of test dataset
test_corpus = []

for i in range(0, len(test)):
  
  # tokenizing the news-title by words
  words = test[i].split()

  # removing the stopwords
  words = [word for word in words if word not in set(stopwords.words('english'))]

  # stemming the words
  words = [ps.stem(word) for word in words]

  # joining the stemmed words
  headline = ' '.join(words)

  # building a corpus of news-title
  test_corpus.append(headline)

In [ ]:
test_corpus[0:1]

In [ ]:
down_words = []
for i in list(y_train[y_train==0].index):
  down_words.append(train_corpus[i])

up_words = []
for i in list(y_train[y_train==1].index):
  up_words.append(train_corpus[i])

In [ ]:
down_words_all = " ".join(down_words)
up_words_all = " ".join(up_words)

In [ ]:
# creating wordcloud for down_words
wordcloud_down = WordCloud(background_color='white', width=3000, height=2500).generate(down_words_all)
plt.figure(figsize=(8,8))
plt.imshow(wordcloud_down)
plt.axis('off')
plt.title("Words which indicate a fall in S&P 500")
plt.show()

In [ ]:
# creating wordcloud for up_words
wordcloud_up = WordCloud(background_color='white', width=3000, height=2500).generate(up_words_all)
plt.figure(figsize=(8,8))
plt.imshow(wordcloud_up)
plt.axis('off')
plt.title("Words which indicate a rise in S&P 500")
plt.show()

In [ ]:
# creating the Bag of Words model
cv = CountVectorizer(max_features=10000, ngram_range=(2,2))
X_train = cv.fit_transform(train_corpus).toarray()

In [ ]:
#test model to append to others features
X_train

In [ ]:
X_test = cv.transform(test_corpus).toarray()

In [ ]:
X_test

**<font color="#5963ab" size="4">macro_data - sp500_rates</font>**

In [ ]:
# to delete later copy to retrieva data if something went wromg
sp500_rates_copy = sp500_rates.copy()
sp500_rates_copy

In [ ]:
# plot all data
sp500_rates_plot = sp500_rates.plot.line( use_index = True)
sp500_rates_plot.set_ylabel("All rates for S&P 500")
sp500_rates_plot.set_title('S&P 500 Rates')
sp500_rates_plot.set_ylim(-10, 50)

In [ ]:
# calculate the correlation matrix
correlation_matrix = sp500_rates.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

>Na matrycy korelacji można zauważyć, że współczynniki P/E oraz Earning Yield są przeciwstawne do siebie, co jest zgodne ze sposobem ich obliczania. Widać także dużą pozytywną korelację pomiędzy współczynnikiami CAPE, P/B, P/S, oraz dużą ujemną korelację pomiędzy tymi wskaźnikami a Dividend Yield.
>
> Ogólnie można zauważyc, że danę nie są bardzo skorelowane ze sobą, co jest korzystne do wykorzystania danych w modealch uczenia maszynowego. Niskie lub brak skorelowania między cechami może pomóc w zrozumieniu, jakie cechy wpływają na predykcje modelu Warto jednak eksperymentować z różnymi użytymi danymi  i monitorować wydajność modelu oraz interpretowalność wyników, aby dostosować podejście do optymalnego rozwiązania problemu.

**<font color="#5963ab" size="4">macro_data - usa_eco_rates</font>**

In [ ]:
# to delete later copy to retrieva data if something went wromg
usa_eco_rates_copy = usa_eco_rates.copy()
usa_eco_rates_copy

In [ ]:
# plot all data without GDP
usa_eco_rates_wo_gdp = usa_eco_rates.copy()
del usa_eco_rates_wo_gdp['gdp']
usa_eco_rates_plot = usa_eco_rates_wo_gdp.plot.line( use_index = True)
usa_eco_rates_plot.set_ylabel("All rates for USA market without GDP")
usa_eco_rates_plot.set_title('USA Economy Rates')

In [ ]:
# calculate the correlation matrix
correlation_matrix = usa_eco_rates.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

> Dane dla amerykańskiej gospodarki nie są bardzo skorelowane ze sobą. Można wyróżnić pozytywna korelację pomiędzy 10 US Bond Yield a EFFR, co jest zgodne z logiką, że rentowności obligacji skarbowych dażą do wartośći efektywnych stóp procentowych. Drugą parą o dodatniej korelacji jest PMI Index i GDP Percent Change. Widać tutaj powiązanie, wzrostu PKB USA w danym okresie czesu jest skorelowane z poztywnym nastawieniem managerów do nabywania różnego rodzaju dóbr i usług.
>
> Największa ujemną korelację jest dla pary EFFR, Unempolyment Rate. Co może być związane z polityką banków centralnych, którzy odpowiednio sterują stopami procentowymi i częstym kryterium  do przyjęcia odpowiedniej strategii jest wartość bezrobocia w kraju. Drugą parą są wskaźniki GDP oraz 10 US Bond Yield. Co może być powiązane szczególnie z czasami krysyzu, w którym to PKB kraju często maleje a inwestorzy inwestują w bezpieczniejsze aktywa jak obligacje.

**<font color="#5963ab" size="4">goog_trend</font>**

In [ ]:
# load the dataset
goog_trend = pd.read_csv('/kaggle/input/s-and-p500-google-trends/SP500_GoogleTrends.csv', index_col='date')
goog_trend.index = pd.to_datetime(goog_trend.index)
goog_trend.index = goog_trend.index.tz_localize('America/New_York')
goog_trend

In [ ]:
# calculate the difference between the current month and the previous month
goog_trend['month_diff'] = goog_trend['month_trend'].diff()
goog_trend

In [ ]:
# plot trend
#goog_trend_plot = goog_trend['month_diff'].plot.line( use_index = True)
goog_trend_plot = goog_trend.plot.line( use_index = True)
goog_trend_plot.set_ylabel("Month change in Trend")
goog_trend_plot.set_title('S&P 500 Changes in trend')

> Należy rozważyć, w projekcie czy warto używać wartości bezwzględnych czy wartości względnych (zmian względem poprzedniego okresu czasu). Oba podejścia mogą być cenne w różnych sytuacjach, dlatego należy rozważyć kilka kwestii, kiedy używać każdego z tych podejść. Oto krótkie charakterystyki podejść:
>
>**1. Wartości Bezwzględne:**
>
> - **Stabilne Trendy:** Jeśli Twoje dane wykazują stałe i spójne trendy w czasie, używanie wartości bezwzględnych może być dobrym wyborem. Na przykład, jeśli przewidujesz ceny akcji, sama absolutna cena może być bardziej informatywna niż zmiana w stosunku do poprzedniego dnia.
> - **Interpretowalność:** Wartości bezwzględne są często bardziej interpretowalne, ponieważ reprezentują rzeczywiste pomiary lub ilości. Może to być istotne w przypadkach, gdy musisz wyjaśnić lub uzasadnić przewidywania przed interesariuszami.
>
>**2. Wartości Względne (Zmiany):**
>
> - **Sezonowość:** Gdy Twoje dane wykazują silne wzorce sezonowe lub cykliczne zachowanie, może być korzystne pracować z wartościami względnymi (zmianami). Na przykład w prognozowaniu sprzedaży może być bardziej informatywne modelowanie zmian miesiąc po miesiącu lub rok po roku niż same liczby sprzedaży.
> - **Stacjonarność:** Niektóre algorytmy uczenia maszynowego zakładają, że dane są stacjonarne, co oznacza, że właściwości statystyczne nie zmieniają się w czasie. W takich przypadkach modelowanie różnic między okresami czasu może pomóc uczynić dane bardziej stacjonarnymi.
> - **Redukcja Szumu:** Modelowanie zmian może pomóc usunąć szum lub nieregularności w danych, co ułatwia modelowi skupienie się na ukrytych wzorcach.
>
>**3. Łączenie obu podjeść:**
>
> - W wielu przypadkach warto pracować zarówno z wartościami bezwzględnymi, jak i wartościami względnymi jako cechy. Na przykład możesz uwzględnić zarówno absolutną wartość zmiennej, jak i jej zmianę względem poprzedniego okresu czasu jako osobne cechy w modelu. To może uwzględnić zarówno długoterminowe trendy, jak i krótkoterminową dynamikę w danych.

In [ ]:
# calculate the correlation matrix
correlation_matrix = goog_trend.corr()

# create a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap")
plt.show()

>Można zauważyć, że jest nieduża korelacja pomiędzy absolutnymi wartościami trendów wyszukiwania a relatywnymi miesięcznymi zmianami dla trendu.

**<font color="#5963ab" size="4">analiza techniczna</font>**

mze tutaj merge wysztkich danych i zrobienie korelacji i innych rzeczy na tym podstawie wybranie ostatcznych danych do modeli

<div id="chap5"><h1 style="color:white;background:#5963ab;border-radius:5px;padding:30px;font-family:'Arial', cursive;font-size:50px;text-align:center">Stworzenie modeli</h1></div>

**<font color="#5963ab" size="4">nyt_hl</font>**

**<font color="#5963ab" size="4">final data</font>**

In [ ]:
# create data frame for storing all preprocessed data
final_data = pd.DataFrame()

In [ ]:
# create column for prediction for next day - True if the index increases the next day, or False if it decreases
sp500_predict = sp500.copy()
sp500_predict["tomorrow"] = sp500_predict["close"].shift(-1)
sp500_predict = sp500_predict.dropna()
sp500_predict["target"] = (sp500_predict["tomorrow"]> sp500_predict["close"]).astype(int)
sp500_predict

In [ ]:
# copy all needed data from sp500 data to final_data
final_data = sp500_predict[['target', 'volume', 'close']]
final_data

>Wnioksi :P
>
>Przewidywanie cen akcji na giełdzie jest od dawna przedmiotem zainteresowania zarówno analityków, jak i badaczy. Ceny akcji są trudne do przewidzenia ze względu na ich dużą zmienność, która zależy od różnorodnych czynników politycznych i ekonomicznych, zmiany przywództwa, nastrojów inwestorów i wielu innych czynników. Przewidywanie cen akcji na podstawie samych danych historycznych lub informacji tekstowych okazało się niewystarczające.